In [1]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
import pandas as pd
import data_pecarn 
import data_psrc
import data
from collections import Counter
plt.style.use('dark_background')
# sns.set(style="white")
%matplotlib inline
%load_ext autoreload
%autoreload 2
NUM_PATIENTS = 12044

In [51]:
dummy = False
df_pecarn = data_pecarn.get_data(use_processed=False,
                                 frac_missing_allowed=0.1,
                                 dummy=dummy)
all_feats_pecarn, filtered_feats_pecarn = data.get_feat_names(df_pecarn)
df_psrc = data_psrc.get_data(use_processed=False, dummy=dummy)
all_feats_psrc, filtered_feats_psrc = data.get_feat_names(df_psrc)

computing pecarn preprocessing...
computing psrc preprocessing...


In [34]:
[k for k in df_psrc if 'skull' in k]

[]

# still need to find all the rel vars for distracting pain

In [50]:
df = df_psrc
df['DistractingPain'] = np.array([False] * df.shape[0])
for k in ['Chest X-ray (choice=Rib fracture)', 'Indicate thoracic injury (choice=Clavicle fracture)']:
    df['DistractingPain'] = df['DistractingPain'] | df[k]
    print(df['DistractingPain'].sum())

47
102


In [ ]:
list(df_psrc.keys())

In [26]:
outcome_def = 'iai_intervention' # output
common_feats = list(filtered_feats_pecarn.intersection(filtered_feats_psrc))
common_feats = common_feats + [outcome_def, 'cv_fold', 'dset']
print(common_feats, len(common_feats))

['AbdDistention', 'Hypotension', 'LtCostalTender', 'RecodedMOI', 'DecrBreathSound', 'Age', 'AbdTrauma_or_SeatBeltSign', 'AbdTrauma', 'GCSScore', 'AbdomenPain', 'InitSysBPRange', 'SeatBeltSign', 'VomitWretch', 'CostalTender', 'RtCostalTender', 'InitHeartRate', 'Race', 'ThoracicTrauma', 'AbdTenderDegree', 'iai_intervention', 'cv_fold', 'dset'] 22


In [ ]:
for feat in common_feats:
    print(feat, df_pecarn[feat].unique(),
          df_pecarn[feat].min(), df_pecarn[feat].max())

**look at resulting feats**

In [42]:
filtered_feats_pecarn.intersection(filtered_feats_psrc)

{'AbdDistention',
 'AbdTenderDegree',
 'AbdTrauma',
 'AbdTrauma_or_SeatBeltSign',
 'AbdomenPain',
 'Age',
 'CostalTender',
 'DecrBreathSound',
 'GCSScore',
 'Hypotension',
 'InitHeartRate',
 'InitSysBPRange',
 'LtCostalTender',
 'Race',
 'RecodedMOI',
 'RtCostalTender',
 'SeatBeltSign',
 'ThoracicTrauma',
 'VomitWretch'}

In [43]:
print('psrc but not pecarn')
filtered_feats_psrc.difference(filtered_feats_pecarn)

psrc but not pecarn


set()

In [45]:
print('pecarn but not psrc')
print(filtered_feats_pecarn.difference(filtered_feats_psrc))
print('(also still missing femur fracture)')

pecarn but not psrc
{'ThoracicTender', 'DistractingPain'}
(also still missing femur fracture)


# look for feat shift

In [ ]:
R, C, = 2, 2
feats = ['InitSysBPRange', 'InitHeartRate', 'GCSScore', 'Age']
for i in range(R * C):
    plt.subplot(R, C, i + 1)
    feat = feats[i]
    plt.hist(df_pecarn[feat], density=True, alpha=0.5, label='pecarn')
    plt.hist(df_psrc[feat], density=True, alpha=0.5, label='')
    plt.xlabel(feat)
plt.tight_layout()
plt.show()

# write csvs

In [23]:
df_pecarn.astype('object').describe().transpose().to_csv('processed/pecarn_vars.csv')
df_psrc.astype('object').describe().transpose().to_csv('processed/psrc_vars.csv')

In [117]:
common_feats = [x for x in feat_names_pecarn if x in feat_names_psrc]

In [17]:
df_psrc['Emesis post injury'].unique()

array([ 0., nan,  1.])